In [2]:
PDF_FILE = "/teamspace/studios/this_studio/Computer Networking A Top-Down Approach, 8th Edition by James F. Kurose, Keith W. Ross-Pearson-9780136681557 (2).pdf"

# We'll be using Llama 3.1 8B for this example.
MODEL = "llama3.1"

In [3]:
!pip install -r requirements.txt

In [4]:

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(PDF_FILE)
pages = loader.load()

print(f"Number of pages: {len(pages)}")
print(f"Length of a page: {len(pages[1].page_content)}")
print("Content of a page:", pages[1].page_content)

Number of pages: 775
Length of a page: 154
Content of a page: James F. Kurose
University of Massachusetts, Amherst
Keith W. Ross
Polytechnic Institute of NYU
COMPUTER
NETWORKING
A Top-Down Approach
www.EBooksWorld.ir


In [5]:

from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)

chunks = splitter.split_documents(pages)
print(f"Number of chunks: {len(chunks)}")
print(f"Length of a chunk: {len(chunks[1].page_content)}")
print("Content of a chunk:", chunks[1].page_content)

Number of chunks: 1759
Length of a chunk: 427
Content of a chunk: xix1
81
181
303377449531Chapter 1 Computer Networks and the Internet 
Chapter 2 Application Layer  
Chapter 3 Transport Layer  
Chapter 4 The Network Layer: Data Plane  
Chapter 5 The Network Layer: Control Plane  
Chapter 6 The Link Layer and LANs  
Chapter 7 Wireless and Mobile Networks  
Chapter 8 Security in Computer Networks  607
References 691
Index 731Brief Contents
A01_KURO1557_08_SE_FM.indd   19 12/02/20   10:57 AM


ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [9]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 182.1 MB/s eta 0:00:0000:0100:01


In [10]:

from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model=MODEL)
vectorstore = FAISS.from_documents(chunks, embeddings)

In [11]:

retriever = vectorstore.as_retriever()
retriever.invoke("What can you get away with when you only have a small number of users?")

[Document(metadata={'source': '/teamspace/studios/this_studio/Computer Networking A Top-Down Approach, 8th Edition by James F. Kurose, Keith W. Ross-Pearson-9780136681557 (2).pdf', 'page': 614}, page_content='In a prior edition of this interview I said that I have never put much faith into predicting the \nfuture, but I did go on to speculate that we might see the end of feature phones (i.e., those that are not programmable and are used only for voice and text messaging) as smart phones become more and more powerful and the primary point of Internet access for many—and now not so many years later that is clearly the case. I also predicted that we would see the continued proliferation of embedded SIMs by which all sorts of devices have the ability to communicate via the cellular network at low data rates. While that has occurred, we see many devices and “Internet of Things” that use embedded WiFi and other lower power, shorter range, forms of connectivity to local hubs. I did not antici

In [13]:
!pip install langchain_ollama

In [14]:

from langchain_ollama import ChatOllama

model = ChatOllama(model=MODEL, temperature=0)
model.invoke("Who is the president of the United States?")

AIMessage(content='As of my last update in April 2023, Joe Biden is the President of the United States. He took office on January 20, 2021, succeeding Donald Trump as the 46th President of the United States. Please note that this information might change over time due to elections or other political developments.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-09-17T17:09:48.780994784Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1638886725, 'load_duration': 22152502, 'prompt_eval_count': 19, 'prompt_eval_duration': 41214000, 'eval_count': 65, 'eval_duration': 1531969000}, id='run-50f3f7af-a7f9-433c-b477-39b562f4a93c-0', usage_metadata={'input_tokens': 19, 'output_tokens': 65, 'total_tokens': 84})

In [15]:

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
print(chain.invoke("Who is the president of the United States?"))

As of my last update in April 2023, Joe Biden is the President of the United States. He took office on January 20, 2021, succeeding Donald Trump as the 46th President of the United States. Please note that this information might change over time due to elections or other political developments.


In [16]:
from langchain.prompts import PromptTemplate

template = """
You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: Here is some context

Question: Here is a question



In [17]:
chain = prompt | model | parser

chain.invoke({
    "context": "Anna's sister is Susan", 
    "question": "Who is Susan's sister?"
})

'Anna.'

In [18]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [1]:
questions = [
    "When are the layers in the internet protocol?",
    "What do you mean by conditional GET?",
    "What is HTTP. Explain?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print("*************************\n")

Question: When are the layers in the internet protocol?


NameError: name 'chain' is not defined